In [57]:
import numpy as np
import pandas as pd
import os
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time

In [58]:
personality_data = pd.read_csv("clean_data_2.csv")

In [59]:
personality_data.head()

,type,is_Extrovert,is_Sensing,is_Thinking,is_Judging,posts,clean_posts,compound_sentiment,pos_sentiment,neg_sentiment,...,ADV_avg,CONJ_avg,DET_avg,NOUN_avg,NUM_avg,PRT_avg,PRON_avg,VERB_avg,._avg,X_avg
0,INFJ,0,0,0,1,'When asked of the things you wish you did ear...,"'when asked thing wish earlier (' ), find answ...",0.999850,0.530909,0.211268,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,INFJ,0,0,0,1,'I love both and they are equally important to...,' love equally important . music window soul. ...,0.999950,0.747273,0.222535,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,INFJ,0,0,0,1,Really? You think implying that everyone who i...,really? think implying everyone entrepreneur s...,0.996749,0.423636,0.439437,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENFJ,1,0,0,1,'Love is a crazy thing. Se is our best form ...,"'love crazy thing. best form communication,...",0.999600,0.532727,0.447887,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,INTP,0,0,1,0,'I am a physics undergrad with a computation e...,"' physic undergrad computation emphasis, /will...",0.999650,0.523636,0.287324,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
personality_data.shape

(8588, 25)

In [61]:
def unique_words(s):
    unique = set(s.split(" "))
    return len(unique) / 50


def emojis(post):
    # does not include emojis made purely from symbols, only :word:
    emoji_count = 0
    words = post.split()
    for e in words:
        if "http" not in e:
            if e.count(":") == 2:
                emoji_count += 1
    return emoji_count / 50


def colons(post):
    # Includes colons used in emojis
    colon_count = 0
    words = post.split()
    for e in words:
        if "http" not in e:
            colon_count += e.count(":")
    return colon_count / 50

In [62]:
personality_data["qm"] = personality_data["posts"].apply(lambda s: s.count("?") / 50)
personality_data["em"] = personality_data["posts"].apply(lambda s: s.count("!") / 50)
personality_data["colons"] = personality_data["posts"].apply(colons)
personality_data["emojis"] = personality_data["posts"].apply(emojis)

In [63]:
personality_data["word_count"] = personality_data["posts"].apply(
    lambda s: (s.count(" ") + 1) / 50
)
personality_data["unique_words"] = personality_data["posts"].apply(unique_words)

In [64]:
# stats

t = time.time()

# personality_data["avg_word_ct"] = personality_data["word_count"].apply(lambda s: s / 50)

personality_data["post_length_var"] = personality_data["posts"].apply(
    lambda x: np.var([len(post.split()) for post in x.split("|||")])
)

print(f"Time Taken: {time.time() - t}")

Time Taken: 0.8633599281311035


In [65]:
personality_data["upper"] = personality_data["posts"].apply(
    lambda x: len([x for x in x.split() if x.isupper()]) / 50
)

In [66]:
personality_data["link_count"] = personality_data["posts"].apply(
    lambda s: s.count("http") / 50
)


In [67]:
ellipses_count = [
    len(re.findall(r"\.\.\.\ ", posts)) / 50 for posts in personality_data["posts"]
]
personality_data["ellipses"] = ellipses_count

In [68]:
personality_data["img_count"] = [
    len(re.findall(r"(\.jpg)|(\.jpeg)|(\.gif)|(\.png)", post)) / 50
    for post in personality_data["posts"]
]

In [69]:
personality_data.head(2)

,type,is_Extrovert,is_Sensing,is_Thinking,is_Judging,posts,clean_posts,compound_sentiment,pos_sentiment,neg_sentiment,...,em,colons,emojis,word_count,unique_words,post_length_var,upper,link_count,ellipses,img_count
0,INFJ,0,0,0,1,'When asked of the things you wish you did ear...,"'when asked thing wish earlier (' ), find answ...",0.99985,0.530909,0.211268,...,0.22,0.32,0.08,30.98,14.92,78.414931,1.46,0.04,0.62,0.0
1,INFJ,0,0,0,1,'I love both and they are equally important to...,' love equally important . music window soul. ...,0.99995,0.747273,0.222535,...,0.36,0.14,0.00,28.58,12.72,160.744400,1.62,0.02,0.04,0.0


In [70]:
personality_data.dtypes

type                   object
is_Extrovert            int64
is_Sensing              int64
is_Thinking             int64
is_Judging              int64
posts                  object
clean_posts            object
compound_sentiment    float64
pos_sentiment         float64
neg_sentiment         float64
neu_sentiment         float64
tag_posts              object
tagged_words           object
ADJ_avg               float64
ADP_avg               float64
ADV_avg               float64
CONJ_avg              float64
DET_avg               float64
NOUN_avg              float64
NUM_avg               float64
PRT_avg               float64
PRON_avg              float64
VERB_avg              float64
._avg                 float64
X_avg                 float64
qm                    float64
em                    float64
colons                float64
emojis                float64
word_count            float64
unique_words          float64
post_length_var       float64
upper                 float64
link_count

In [71]:
personality_data.isnull().sum()

type                  0
is_Extrovert          0
is_Sensing            0
is_Thinking           0
is_Judging            0
posts                 0
clean_posts           0
compound_sentiment    0
pos_sentiment         0
neg_sentiment         0
neu_sentiment         0
tag_posts             0
tagged_words          0
ADJ_avg               0
ADP_avg               0
ADV_avg               0
CONJ_avg              0
DET_avg               0
NOUN_avg              0
NUM_avg               0
PRT_avg               0
PRON_avg              0
VERB_avg              0
._avg                 0
X_avg                 0
qm                    0
em                    0
colons                0
emojis                0
word_count            0
unique_words          0
post_length_var       0
upper                 0
link_count            0
ellipses              0
img_count             0
dtype: int64

In [72]:
personality_data.to_csv(("clean_data_3.csv"), index=False)

In [73]:
tfidf_vectorizer = TfidfVectorizer(min_df=25, max_df=0.8)
tfidf_words = tfidf_vectorizer.fit_transform(personality_data["clean_posts"])
tfidf_vectorized_data = pd.DataFrame(
    data=tfidf_words.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
)


In [74]:
tfidf_vectorized_data.head()

,000,100,1000,101,10char,10th,110,11th,120,125,...,zen,zero,zodiac,zombie,zombies,zone,zoned,zoning,zoo,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
tfidf_vectorized_data.to_csv(("tfidf_vectorized_data.csv"), index=False)

In [76]:
count_vectorizer = CountVectorizer(decode_error="ignore", min_df=25, max_df=0.8)
count_words = count_vectorizer.fit_transform(personality_data["clean_posts"])
count_vectorized_data = pd.DataFrame(
    data=count_words.toarray(), columns=count_vectorizer.get_feature_names_out()
)


In [77]:
count_vectorized_data.head()

,000,100,1000,101,10char,10th,110,11th,120,125,...,zen,zero,zodiac,zombie,zombies,zone,zoned,zoning,zoo,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
count_vectorized_data.to_csv(("count_vectorized_data.csv"), index=False)